# Imports

In [832]:
import json, os
from graphviz import Digraph

# NFA Creator

In [833]:
EPSILON = "ε" #ε
PLUS = '+'
STAR = '*'
Q_MARK = '?'
OR = '|'

ALL_LETTERS = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
ALL_NUMBERS = "1234567890"
ALL_SPECIAL = " ~[{]}!@#-_+=$%^&*()?><"

ALL_CHARACTERS = ALL_LETTERS+ALL_NUMBERS+ALL_SPECIAL

def firstElementValidation(regex):
    if len(regex) > 0:
        c = regex[0]
        if c == PLUS or c == STAR or c == '/' or c == Q_MARK:
            return False
    return True

def regexBracketValidation(regex):
    all_brackets = []
    index = 0
    for c in regex:
        back_iterator = index - 1
        index += 1
        backslash_count = 0
        while back_iterator >= 0 and regex[back_iterator] == '\\':
            backslash_count += 1
            back_iterator -= 1
        if backslash_count % 2 == 1:
            continue
        if c == '(':
            flag = False
            for b in all_brackets:
                if b == '[':
                    flag = True
                    break
            if flag:
                continue
            all_brackets.append(c)
        elif c == ')':
            flag = False
            for b in all_brackets:
                if b == '[':
                    flag = True
                    break
            if flag:
                continue
            if len(all_brackets) > 0:
                if all_brackets[-1] == '(':
                    all_brackets.pop()
                else:
                    return False
            else:
                return False
        elif c == '[':
            flag = False
            for b in all_brackets:
                if b == '[':
                    flag = True
                    break
            if flag:
                continue
            all_brackets.append(c)
        elif c == ']':
            if len(all_brackets) > 0:
                if all_brackets[-1] == '[':
                    all_brackets.pop()
                else:
                    return False
        elif c == '{':
            all_brackets.append(c)
        elif c == '}':
            if len(all_brackets) > 0:
                if all_brackets[-1] == '{':
                    all_brackets.pop()
                else:
                    return False
            else:
                return False
        elif c == Q_MARK or c == STAR or c == PLUS:
            if (len(all_brackets) > 0 and all_brackets[-1] == '[') or index == 0:
                continue
            elif regex[index - 2] == Q_MARK or regex[index - 2] == STAR or regex[index - 2] == PLUS:
                return False

    if len(all_brackets) > 0:
        return False
    return True

def lastBackslashValidation(regex):
    if regex[-1] == '\\':
        back_iterator = len(regex) - 2
        backslash_count = 0
        while back_iterator >= 0 and regex[back_iterator] == '\\':
            backslash_count += 1
            back_iterator -= 1
        return backslash_count % 2 == 1
    return True

def squareBracketContentsValidation(regex):
    if len(regex) == 0 or (len(regex) == 1 and regex[0] == '^'):
        return False
    index = 0
    dash_indices = []
    for c in regex:
        if index == 1 and c == '-' and regex[0] == '^':
            index += 1
            continue
        if c == '-' and index > 0 and index < len(regex) - 1:
            if len(dash_indices) > 0:
                old_index = dash_indices[-1]
                if index == old_index + 1 or index == old_index + 2:
                    index += 1
                    continue
            dash_indices.append(index)
        index += 1
    for dash_index in dash_indices:
        if regex[dash_index - 1] > regex[dash_index + 1]:
            return False
    return True

def bracketContentsValidation(regex):
    index = 0
    bracket_indices_stack = []

    for c in regex:
        if c == '(':
            found_opening_bracket = False
            for b in bracket_indices_stack:
                if regex[b] == '[':
                    found_opening_bracket = True
                    break
            if found_opening_bracket:
                continue
            bracket_indices_stack.append(index)
        elif c == ')':
            found_opening_bracket = False
            for b in bracket_indices_stack:
                if regex[b] == '[':
                    found_opening_bracket = True
                    break
            if found_opening_bracket:
                continue
            opening_index = bracket_indices_stack.pop()
            if not firstElementValidation(regex[opening_index+1:index]):
                return False
        elif c == '[':
            found_opening_bracket = False
            for b in bracket_indices_stack:
                if regex[b] == '[':
                    found_opening_bracket = True
                    break
            if found_opening_bracket:
                continue
            bracket_indices_stack.append(index)
        elif c == ']':
            found_opening_bracket = False
            for b in bracket_indices_stack:
                if regex[b] == '[':
                    found_opening_bracket = True
                    break
            if not found_opening_bracket:
                continue
            opening_index = bracket_indices_stack.pop()
            if not squareBracketContentsValidation(regex[opening_index+1:index]):
                return False
        index += 1
    return True

def validateRegex(regex):
    if len(regex) == 0:
        return True
    if not firstElementValidation(regex):
        return False
    if not regexBracketValidation(regex):
        return False
    if not lastBackslashValidation(regex):
        return False
    if not bracketContentsValidation(regex):
        return False
    return True

def lexSquareBrackets(regex, level):
    is_allowed = True
    allowed_characters = []
    index = 0
    dash_indices = []
    for c in regex:
        if index == 1 and c == '-' and regex[0] == '^':
            index += 1
            continue
        if c == '-' and index > 0 and index < len(regex) - 1:
            if len(dash_indices) > 0:
                old_index = dash_indices[-1]
                if index == old_index + 1 or index == old_index + 2:
                    index += 1
                    continue
            dash_indices.append(index)
        index += 1
    if regex[0] == '^':
        is_allowed = False

    allowed_tuples = []
    for dash_index in dash_indices:
        current_tuple = (regex[dash_index - 1], regex[dash_index + 1])
        if current_tuple not in allowed_tuples:
            allowed_tuples.append(current_tuple)

    allowed_all = []

    index = -1
    dash_index = 0
    for c in regex:
        index += 1
        if index == 0 and regex[0] == '^':
            continue
        if dash_index < len(dash_indices):
            if index == dash_indices[dash_index] or index == dash_indices[dash_index] - 1:
                continue
            elif index == dash_indices[dash_index] + 1:
                dash_index += 1
                continue
        add_c = True
        for t in allowed_tuples:
            if c >= t[0] and c <= t[1]:
                add_c = False
        if add_c and c not in allowed_characters:
            allowed_characters.append(c)

    for c in allowed_characters:
        allowed_all.append(c)
    for t in allowed_tuples:
        allowed_all.append(t)
    
    # print(allowed_all)
    
    return (allowed_all, level, is_allowed)

# global character_level_extra
# character_level_extra = []

def lexBrackets(regex, level = 0):
    i = -1
    last_ignorer = 0
    indices = []
    for c in regex:
        i += 1
        if i < last_ignorer:
            continue
        if c == '[':
            indices.append(i)
            character_level_extra.append((c, level, None))
            brackets_indices = [i]
            for j in range(i + 1, len(regex)):
                if regex[j] == ']':
                    brackets_indices.pop()
                    if len(brackets_indices) == 0:
                        character_level_extra.append(lexSquareBrackets(regex[i+1:j], level + 1))
                        last_ignorer = j
                        break
        elif c == '(':
            indices.append(i)
            character_level_extra.append((c, level, None))
            brackets_indices = [i]
            for j in range(i + 1, len(regex)):
                if regex[j] == '(':
                    brackets_indices.append(j)
                elif regex[j] == ')':
                    brackets_indices.pop()
                    if len(brackets_indices) == 0:
                        print(regex[i+1:j])
                        lexBrackets(regex[i+1:j], level + 1)
                        last_ignorer = j
                        break
        elif c == Q_MARK or c == STAR or c == PLUS:
            continue
        else:
            if c == '\\':
                last_ignorer += 2
            if not(c == ')' or c == ']'):
                # print(regex[i])
                pass
            extra = None
            if i + 1 < len(regex):
                next = regex[i + 1]
                if next == Q_MARK or next == STAR or next == PLUS:
                    extra = next
                elif regex[i] == '\\':
                    extra = next
                    if i + 2 < len(regex):
                        next_of_next = regex[i + 2]
                        if next_of_next == Q_MARK or next_of_next == STAR or next_of_next == PLUS:
                            extra += next_of_next
            character_level_extra.append((c, level, extra))
            indices.append(i)
    characters = []
    for j in indices:
        characters.append(regex[j])
    # print(regex, characters)
    # print(regex, indices)
    return character_level_extra

def removeUnnessecaryBrackets(character_level_extra):
    while True:
        found = False
        index = -1
        for c_l_e in character_level_extra:
            index += 1
            character = c_l_e[0]
            level = c_l_e[1]
            if character == '(':
                for sub_index in range(index, len(character_level_extra)):
                    if character_level_extra[sub_index][0] == ')' and character_level_extra[sub_index][1] == level:
                        if character_level_extra[sub_index][2] == None:
                            if index > 0 and character_level_extra[index - 1][2] != OR and character_level_extra[index - 1][0] != OR:
                                # print('Removing:', c_l_e, character_level_extra[sub_index], index, sub_index)
                                found = True
                                character_level_extra.pop(sub_index)
                                character_level_extra.pop(index)
                        break

        if not found:
            break
    return character_level_extra

def fillEmptyBrackets(character_level_extra):
    index = -1
    for c_l_e in character_level_extra:
        index += 1
        character = c_l_e[0]
        level = c_l_e[1]
        if character == '(':
            if index + 1 < len(character_level_extra) and character_level_extra[index + 1][0] == ')' and level == character_level_extra[index + 1][1]:
                character_level_extra.insert(index + 1, (EPSILON, level + 1, None))
    return character_level_extra

def fillEmptyOrsRight(character_level_extra):
    index = -1
    for c_l_e in character_level_extra:
        index += 1
        character = c_l_e[0]
        level = c_l_e[1]
        if character == OR:
            if index == 0 or character_level_extra[index - 1][0] == '(':
                character_level_extra.insert(index, (EPSILON, level, None))
            elif index == len(character_level_extra) - 1 or character_level_extra[index + 1][0] == ')' or character_level_extra[index + 1][0] == '|':
                character_level_extra.insert(index + 1, (EPSILON, level, None))
            
    return character_level_extra

# State: (name, isTerminatingState, [(input, State)]
global states
states = []

def lastOr(character_level_extra, state_indexer):
    index = -1
    for c_l_e in character_level_extra:
        index += 1
        if index < state_indexer :
            continue
        if c_l_e[0] == OR:
            return False
    return True
   
def createStates(character_level_extra, state_index = 1, previous_state_index = 0, next_state_index = None):
    base_call = False
    to_be_connected = []
    first_or = True
    first_or_previous = None
    last_or = False
    if next_state_index is None:
        base_call = True
        states.clear()
        states.append(('S0', False, []))
        absolute_previous_index = 0
    else:
        absolute_next_index = next_state_index
        absolute_previous_index = previous_state_index

    skipper = 0
    state_indexer = -1
    or_flag = None
    for c_l_e in character_level_extra:
        state_indexer += 1
        if skipper > 0:
            skipper -= 1
            continue

        # in-state
        previous_state = states[previous_state_index]

        character, level, extra = c_l_e

        # Create the new state to be next (out-state)
        if next_state_index is None or previous_state_index == next_state_index:
            next_state_index = state_index
            new_state_name = 'S'+str(state_index)
            new_state = (new_state_name, False, [])
            states.append(new_state)
            next_state = states[state_index]

            state_index += 1

            if or_flag == True:
                # print('\nFound OR at index', state_indexer - 1)
                # print('Previous state index:', previous_state_index)
                # print('Current 1 index:', current_state1)
                # print('Current 2 index:', current_state2)
                # print('Next state index:', next_state_index)
                # print('Absolute Previous index:', absolute_previous_index)
                if first_or:
                    print('First Or at', state_indexer)
                    first_or = False
                    new_state_name = 'S'+str(state_index)
                    new_state = (new_state_name, False, [(EPSILON, first_or_previous)])
                    print('Connecting', new_state_name, 'to', first_or_previous)
                    # print('Trial instead', first_or_previous)
                    states.append(new_state)
                    state_index += 1
                    
                    states[absolute_previous_index][2].clear()
                    states[absolute_previous_index][2].append((EPSILON, new_state_name))
                states[absolute_previous_index][2].append((EPSILON,'S'+str(previous_state_index)))

                if lastOr(character_level_extra, state_indexer):
                    # print('Last Or at', state_indexer)
                    last_or = True
                else:
                    # print('NOT last Or at', state_indexer)
                    last_or = False
        else:
            # Hereeeeee
            next_state = states[next_state_index]


        or_flag = None
        if character == '[':
            skipper = 1
            available_index = -1
            characters = ''
            for c_t in character_level_extra[state_indexer + 1][0]:
                available_index += 1
                add_comma = False
                if available_index < len(character_level_extra[state_indexer + 1][0]) - 1:
                    add_comma = True
                if type(c_t) is str:        # Character
                    characters += c_t
                elif type(c_t) is tuple:    # Tuple
                    characters += (c_t[0] + '-' + c_t[1])
                if add_comma:
                    characters += ', '
            # Create the new states
            new_state_name = 'S'+str(state_index)
            new_state = (new_state_name, False, [])
            states.append(new_state)
            current_state1 = states[state_index]
            state_index += 1

            new_state_name = 'S'+str(state_index)
            new_state = (new_state_name, False, [])
            states.append(new_state)
            current_state2 = states[state_index]
            state_index += 1

            if character_level_extra[state_indexer + 1][2] == False:
                characters = "NOT " + characters

            previous_state[2].append((EPSILON, current_state1[0]))
            current_state1[2].append((characters, current_state2[0]))
            current_state2[2].append((EPSILON, next_state[0]))

            if first_or_previous is None:
                first_or_previous = current_state1[0]

        elif character == '(':
            for iterator in range(state_indexer, len(character_level_extra)):
                current_character = character_level_extra[iterator][0]
                current_level = character_level_extra[iterator][1]
                current_extra = character_level_extra[iterator][2]
                if current_character == ')' and level == current_level:
                    skipper = iterator - state_indexer
                    # print('Call', previous_state_index, next_state_index)
                    c1_index, c2_index, state_index, first_or_local = createStates(character_level_extra[state_indexer + 1:iterator], state_index, previous_state_index, next_state_index)
                    if first_or_previous is None:
                        first_or_previous = first_or_local
                    # print('Return', c1_index, c2_index)
                    current_state1 = states[c1_index]
                    current_state2 = states[c2_index]
                    next_state = states[c2_index]
                    next_state_index = c2_index
                    # print(previous_state, current_state1, current_state2, next_state)
                    extra = current_extra
                    break
                    
        elif character == ']' or character == ')':
            pass
        elif character == OR:
            print('Appending', previous_state_index)
            to_be_connected.append(previous_state_index)
            or_flag = True
        else:
            if character == '\\':
                skipper = 1
                if len(extra) == 1:
                    character += extra
                    extra = None
                else:
                    character += extra[0]
                    extra = extra[1]
            if character == '.':
                character = "CHAR"
                character = '.'
            elif character == '\\w':
                character = "ALPHANUM"
            elif character == '\\W':
                character = "SPECIAL"
            elif character == '\\d':
                character = "DIGIT"
            elif character == '\\D':
                character = "NON-DIGIT"
            # Create the new current states
            new_state_name = 'S'+str(state_index)
            new_state = (new_state_name, False, [])
            states.append(new_state)
            current_state1 = states[state_index]

            state_index += 1

            new_state_name = 'S'+str(state_index)
            new_state = (new_state_name, False, [])
            states.append(new_state)
            current_state2 = states[state_index]

            state_index += 1

            previous_state[2].append((EPSILON, current_state1[0]))
            current_state1[2].append((character, current_state2[0]))
            current_state2[2].append((EPSILON, next_state[0]))

            if first_or_previous is None:
                print('Setting first previous with', current_state1)
                first_or_previous = current_state1[0]
            
        if character != '(':
            if extra == STAR:
                current_state2[2].append((EPSILON, current_state1[0]))
                current_state1[2].append((EPSILON, next_state[0]))
            elif extra == PLUS:
                current_state2[2].append((EPSILON, current_state1[0]))
            elif extra == Q_MARK:
                current_state1[2].append((EPSILON, next_state[0]))
        else:
            print('Previous State:', previous_state)
            print('Current 1 State:', current_state1)
            print('Current 2 State:', current_state2)
            print('Next State:', next_state)
            if extra == STAR:
                current_state2[2].append((EPSILON, current_state1[0]))
                current_state1[2].append((EPSILON, next_state[0]))
            elif extra == PLUS:
                current_state2[2].append((EPSILON, current_state1[0]))
            elif extra == Q_MARK:
                current_state1[2].append((EPSILON, next_state[0]))
        
        if character == '(':
            extra = current_extra
            # print('Here extra character', character, current_extra)

        if character != '[' and state_indexer < len(character_level_extra) - 1:
            # print('Proceeding Here', c_l_e, character_level_extra[state_indexer + 1], previous_state_index, next_state_index)
            previous_state_index = next_state_index        

    if base_call and last_or and next_state_index is not None:
        new_state_name = 'S'+str(state_index)
        new_state = (new_state_name, False, [])
        states.append(new_state)
        states[next_state_index][2].append((EPSILON, new_state_name))
        # print('Last Or, connecting', states[next_state_index][0], 'to', new_state_name)
        next_state_index = state_index
        
        state_index += 1

    if base_call and state_indexer == len(character_level_extra) - 1:
        if next_state_index is None:
            next_state_index = 0
        t_state = states[next_state_index]
        states[next_state_index] = (t_state[0], True, t_state[2])
        print('Terminating State:', states[next_state_index],'\n')

        for state_number in to_be_connected:
            print('At the end, connecting', states[state_number][0], 'to', t_state[0])
            states[state_number][2].append((EPSILON, t_state[0]))

    if not base_call and state_indexer == len(character_level_extra) - 1:
        for state_number in to_be_connected:
            if state_number == absolute_next_index:
                continue
            print('At the end, connecting', states[state_number][0], 'to', absolute_next_index)
            states[state_number][2].append((EPSILON, 'S'+str(absolute_next_index)))


    if not base_call:
        next_state[2].append((EPSILON, 'S'+str(next_state_index)))
        print('Final Connecting', next_state[0], 'to', next_state_index)
        print('Returning with current state 1:', current_state1)
        print('Returning with current state 2:', current_state2)
        return absolute_previous_index, next_state_index, state_index, first_or_previous
    else:
        index = -1
        for state in states:
            index += 1
            state = (state[0], state[1], list(dict.fromkeys(state[2])))
            for input_next in state[2]:
                if input_next[1] == state[0] and input_next[0] == EPSILON:
                    # print('Removing', input_next)
                    state[2].remove(input_next)
            states[index] = state
    
def writeNfa(states):
    open('NFA.json', 'w').close()

    with open('NFA.json', 'w', encoding='utf-8') as f:
        json_string = ""
        dict_to_write = {"startingState": "S0"}
        for state in states:
            sub_dict = {}
            sub_dict["isTerminatingState"] = state[1]
            for input_output in state[2]:
                if input_output[0] not in sub_dict:
                    sub_dict[input_output[0]] = []
                sub_dict[input_output[0]].append(input_output[1])
            dict_to_write[state[0]] = sub_dict
        json_string = json.dumps(dict_to_write, indent=3)
        f.write(json_string)

def drawNfa(view_graph):
    nfa_graph = Digraph(graph_attr={'rankdir': 'LR'})
    with open('NFA.json', 'r', encoding='utf-8') as f:
        data = json.load(f)

    for state in data:
        if state == "startingState":
            continue
        if data[state]["isTerminatingState"] == False:
            nfa_graph.attr("node", shape = 'circle')
        else:
            nfa_graph.attr("node", shape = 'doublecircle')
        nfa_graph.node(state)
        if state == data["startingState"]:
            nfa_graph.attr("node", shape = 'none')
            nfa_graph.node('')
            nfa_graph.edge("", state)
    for state in data:
        if state == "startingState":
            continue
        for edges in data[state]:
            if edges == "isTerminatingState":
                continue
            for edge in data[state][edges]:
                nfa_graph.edge(state, edge, edges)

    picFile = "NFA"
    nfa_graph.render(picFile, view = view_graph, format = 'png', overwrite_source = True)
    os.remove("NFA")

def nfaFlow(view_graph = False):
    input_regex = input("\nEnter regular expression: ")
    if validateRegex(input_regex):
        print('\nValid regex:', input_regex, '\n')
        global character_level_extra
        character_level_extra = []
        character_level_extra = lexBrackets(input_regex)

        character_level_extra = fillEmptyOrsRight(character_level_extra)
        character_level_extra = removeUnnessecaryBrackets(character_level_extra)
        character_level_extra = fillEmptyBrackets(character_level_extra)
        print('Characters to parse:', character_level_extra, '\n')

        createStates(character_level_extra)
        print('NFA states: ',states, '\n')

        writeNfa(states)
        drawNfa(view_graph)

        print('Input regex: ', input_regex + '\n')
        return True   
    else:
        print('Invalid regex')
        return False

# DFA Creator

In [834]:
def getNfaStates():
    with open('NFA.json', 'r', encoding='utf-8') as f:
        data = json.load(f)

    states = {}
    for state in data:
        if state == "startingState":
            continue
        input_next = []
        for edges in data[state]:
            if edges == "isTerminatingState":
                continue
            for edge in data[state][edges]:
                input_next.append((edges, edge))
        states[state] = (data[state]["isTerminatingState"], input_next)

    return states

# Again, not always working algorith; it only works if every input character is unique, so a new function is added
def epsilonlessNfaStates(nfa_states):
    possible_epsiloness_states = {}

    for state in nfa_states:
        current_state_to_be = [state]
        index = -1
        while index < len(nfa_states) - 1:
            index += 1
            for current_state in current_state_to_be:
                for input_next in nfa_states[current_state][1]:
                    if input_next[0] == EPSILON and input_next[1] not in current_state_to_be:
                        current_state_to_be.append(input_next[1])

        current_state_to_be.sort()
        possible_epsiloness_states[state] = current_state_to_be

    start_states = ['S0']
    previous_input = {'S0': None}
    for state in nfa_states:
        for input_next in nfa_states[state][1]:
            if input_next[0] != EPSILON and input_next[1] not in start_states:
                start_states.append(input_next[1])
                previous_input[input_next[1]] = (state, input_next[0])

    to_be_deleted = []
    epsilonless_states = {}
    for state in possible_epsiloness_states:
        if state not in start_states:
            to_be_deleted.append(state)
        else:
            epsilonless_states[state] = (False, [])
    for state in to_be_deleted:
        del possible_epsiloness_states[state]
    
    for state in previous_input:
        is_terminating = False
        for sub_state in possible_epsiloness_states[state]:
            if nfa_states[sub_state][0] == True:
                is_terminating = True
                break
        epsilonless_states[state] = (is_terminating, epsilonless_states[state][1])
        if previous_input[state] is None:
            continue
        for mother_state in epsilonless_states:
            if previous_input[state][0] in possible_epsiloness_states[mother_state]:
                # print('Adding', (previous_input[state][1], state), 'to', mother_state)
                epsilonless_states[mother_state][1].append((previous_input[state][1], state))

    return epsilonless_states

def createDfaStates(nfa_states):
    is_already_dfa = True
    for state in nfa_states:
        possible_inputs = []
        for input_next in nfa_states[state][1]:
            if input_next[0] in possible_inputs:
                is_already_dfa = False
                break
            possible_inputs.append(input_next[0])
        if not is_already_dfa:
            break
    if is_already_dfa:
        print('\nNFA is already a DFA\n')
        return nfa_states
    # print('\nNFA states:', nfa_states, '\n')

    possible_inputs = set()
    for state in nfa_states:
        for input_next in nfa_states[state][1]:
            possible_inputs.add(input_next[0])
    # print('Possible inputs:', possible_inputs, '\n')

    explored_sets_of_states = []

    dfa_states = {'S0': (nfa_states['S0'][0], [])}

    for possible_input in possible_inputs:
        current_state_to_be = ['S0']
        index = -1
        while index < len(nfa_states) - 1:
            index += 1
            for current_state in current_state_to_be:
                for input_next in nfa_states[current_state][1]:
                    if input_next[0] == possible_input and input_next[1] not in current_state_to_be:
                        current_state_to_be.append(input_next[1])
        current_state_to_be.sort()
        current_state_to_be.remove('S0')
        if current_state_to_be not in explored_sets_of_states:
            explored_sets_of_states.append(current_state_to_be)
        dfa_states['S0'][1].append((possible_input, current_state_to_be))

    set_of_new_states = {}

    explored_state_index = 0
    for states in explored_sets_of_states:
        explored_state_index += 1
        set_of_new_states['S'+str(explored_state_index)] = states

    state_index = 0
    for set_of_states in explored_sets_of_states:
        state_index += 1
        is_terminating = False
        for state in set_of_states:
            if nfa_states[state][0] == True:
                is_terminating = True
        
        list_of_transitions = []
        for possible_input in possible_inputs:
            current_state_to_be = []
            states_to_loop_on = []
            states_to_loop_on.extend(set_of_states)

            for current_state in states_to_loop_on:
                for input_next in nfa_states[current_state][1]:
                    if input_next[0] == possible_input and input_next[1] not in current_state_to_be:
                        current_state_to_be.append(input_next[1])

            current_state_to_be.sort()
            if current_state_to_be not in explored_sets_of_states:
                explored_state_index += 1
                explored_sets_of_states.append(current_state_to_be)
                set_of_new_states['S'+str(explored_state_index)] = current_state_to_be
            if len(current_state_to_be) > 0:
                for state_name in set_of_new_states:
                    if set_of_new_states[state_name] == current_state_to_be:
                        list_of_transitions.append((possible_input, state_name))
                        break
        dfa_states['S'+str(state_index)] = (is_terminating, list_of_transitions)

    # Renaming states of start state:
    new_names = []
    for input_next in dfa_states['S0'][1]:
        for state_name in set_of_new_states:
            if set_of_new_states[state_name] == input_next[1]:
                new_names.append((input_next[0], state_name))
                break
    dfa_states['S0'] = (dfa_states['S0'][0], new_names)

    return dfa_states

def drawDfa(dfa_states, view_graph = False, another_name = 'DFA'):
    dfa_graph = Digraph(graph_attr={'rankdir': 'LR'})

    index = 0
    for state in dfa_states:
        if dfa_states[state][0] == False:
            dfa_graph.attr("node", shape = 'circle')
        else:
            dfa_graph.attr("node", shape = 'doublecircle')
        dfa_graph.node(state)
        if index == 0:
            index += 1
            dfa_graph.attr("node", shape = 'none')
            dfa_graph.node('')
            dfa_graph.edge("", state)
    for state in dfa_states:
        for input_next in dfa_states[state][1]:
            dfa_graph.edge(state, input_next[1], input_next[0])

    picFile = another_name
    dfa_graph.render(picFile, view = view_graph, format = 'png', overwrite_source = True)
    os.remove(another_name)

def dfaFlow(view_graph = False):
    nfa_states = getNfaStates()
    epsilonless_nfa_states = epsilonlessNfaStates(nfa_states)
    print('Epsilonless states:', epsilonless_nfa_states)
    drawDfa(epsilonless_nfa_states, another_name = 'Epsilonless_NFA')
    dfa_states = createDfaStates(epsilonless_nfa_states)
    print('\nDfa states:', dfa_states)
    drawDfa(dfa_states, view_graph)
    return dfa_states

# Dfa Minimizer

In [835]:
def minimizeDfa(dfa_states):
    possible_inputs = []
    for state in dfa_states:
        for input_next in dfa_states[state][1]:
            if input_next[0] not in possible_inputs:
                possible_inputs.append(input_next[0])

    accepting_states = []
    non_accepting_states = []


    for state in dfa_states:
        if dfa_states[state][0] == True:
            accepting_states.append(state)
        else:
            non_accepting_states.append(state)

    groups_to_process = []
    if len(accepting_states) > 0:
        groups_to_process.append(accepting_states)
    if len(non_accepting_states) > 0:
        groups_to_process.append(non_accepting_states)

    print('\nInitially non-accepting states:', accepting_states)
    print('\nInitially accepting states:', non_accepting_states)

    while True:
        breaker = True

        copy_to_process = groups_to_process.copy()

        index = -1
        for group in copy_to_process:
            index += 1
            if len(group) > 1:
                out_liers = []
                out_lier_behavior = None
                for possible_input in possible_inputs:
                    elements_behavior = []
                    for element in group:
                        next_state = None
                        for input_next in dfa_states[element][1]:
                            if input_next[0] == possible_input:
                                next_state = input_next[1]
                                break
                        if next_state is not None:
                            sub_index = -1
                            for g in copy_to_process:
                                sub_index += 1
                                if next_state in g:
                                    elements_behavior.append(sub_index)
                                    break
                        else:
                            elements_behavior.append(-1)
                    # print('\nElements behaviour:', elements_behavior, 'for input', possible_input)

                    frequency_dict = {}
                    for behavior in elements_behavior:
                        if behavior not in frequency_dict:
                            frequency_dict[behavior] = 1
                        else:
                            frequency_dict[behavior] = frequency_dict[behavior] + 1

                    minimum_behavior = len(dfa_states)
                    if len(frequency_dict) != 1:
                        for behavior in elements_behavior:
                            if frequency_dict[behavior] < minimum_behavior:
                                minimum_behavior = frequency_dict[behavior]
                    
                        for behavior in elements_behavior:
                            if frequency_dict[behavior] == minimum_behavior:
                                out_lier_behavior = behavior
                                break
                    if out_lier_behavior is not None:
                        sub_index = -1
                        for behavior in elements_behavior:
                            sub_index += 1
                            if behavior == out_lier_behavior:
                                out_liers.append(group[sub_index])
                        # print('\nElements:', out_liers, 'does not belong to group', group)
                        breaker = False
                        break
            if not breaker:
                for out_lier in out_liers:
                    group.remove(out_lier)
                copy_to_process[index] = group
                copy_to_process.append(out_liers)
                # print('\nNew groups:', copy_to_process)
                break

        if breaker:
            break

        groups_to_process = copy_to_process
        
    print('\nFinal groups:', groups_to_process)

    for group in groups_to_process:
        if len(group) > 1:
            if 'S0' in group:
                representative = 'S0'
            else:
                representative = group[0]
            for state in dfa_states:
                index = -1
                for input_next in dfa_states[state][1]:
                    index += 1
                    if input_next[1] in group:
                        dfa_states[state][1][index] = (dfa_states[state][1][index][0], representative)
            for element in group:
                if element == representative:
                    continue
                del dfa_states[element]
    print('\nMinimized Dfa:', dfa_states)
    return dfa_states

def writeMinimizedDfa(dfa_states):
    with open('Minimized_DFA.json', 'w', encoding='utf-8') as f:
        dict_to_write = {"startingState": "S0"}
        for state in dfa_states:
            sub_dict = {}
            sub_dict["isTerminatingState"] = dfa_states[state][0]
            for input_next in dfa_states[state][1]:
                sub_dict[input_next[0]] = input_next[1]
            dict_to_write[state] = sub_dict
        json_string = json.dumps(dict_to_write, indent=3)
        f.write(json_string)

def minimizedDfaFlow(dfa_states):
    minimized_dfa = minimizeDfa(dfa_states)
    drawDfa(minimized_dfa, another_name='Minimized_DFA')
    writeMinimizedDfa(minimized_dfa)        

# Main Flow

In [836]:
def allFlow():
    if nfaFlow() == True:
        minimizedDfaFlow(dfaFlow())
allFlow()


Valid regex: (a|b)c 

a|b
Characters to parse: [('(', 0, None), ('a', 1, None), ('|', 1, None), ('b', 1, None), (')', 0, None), ('c', 0, None)] 

Setting first previous with ('S2', False, [('a', 'S3')])
Appending 1
First Or at 2
Connecting S6 to S2
Returning with current state 1: ('S7', False, [('b', 'S8')])
Returning with current state 2: ('S8', False, [('ε', 'S5')])
Previous State: ('S0', False, [('ε', 'S6'), ('ε', 'S4')])
Current 1 State: ('S0', False, [('ε', 'S6'), ('ε', 'S4')])
Current 2 State: ('S5', False, [('ε', 'S5')])
Next State: ('S5', False, [('ε', 'S5')])
Terminating State: ('S9', True, []) 

NFA states:  [('S0', False, [('ε', 'S6'), ('ε', 'S4')]), ('S1', False, []), ('S2', False, [('a', 'S3')]), ('S3', False, [('ε', 'S1')]), ('S4', False, [('ε', 'S7')]), ('S5', False, [('ε', 'S10')]), ('S6', False, [('ε', 'S2')]), ('S7', False, [('b', 'S8')]), ('S8', False, [('ε', 'S5')]), ('S9', True, []), ('S10', False, [('c', 'S11')]), ('S11', False, [('ε', 'S9')])] 

Input regex:  (a